# Лабораторная работа №1 
## Извлечение признаков из текстовых данных

## Импорты

In [1]:
import re
from tqdm.notebook import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from num2words import num2words
import pandas as pd
from pymorphy3 import MorphAnalyzer
from sklearn.decomposition import PCA

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Azerty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Azerty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Azerty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Загрузка датасета

In [17]:
petitions = pd.read_csv("../datasets/general/petitions.csv")
petitions.head()

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство


In [18]:
df_corpus = petitions["public_petition_text"]
df_corpus.name = "corpus"
display(df_corpus[:5], df_corpus.shape)

0                                       снег на дороге
1                  очистить кабельный киоск от рекламы
2    Просим убрать все деревья и кустарники, которы...
3    Неудовлетворительное состояние парадной - надп...
4                                             Граффити
Name: corpus, dtype: object

(59889,)

## Предобработка

In [19]:
df_corpus.duplicated().sum()

np.int64(16749)

In [20]:
df_corpus = df_corpus.drop_duplicates()
df_corpus.shape

(43140,)

In [21]:
def replace_number(number) -> str:
    return " " + num2words(number.group(0), lang="ru") + " "


def replace_several_words(text: str, replacements: dict[str, str]) -> str:
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    return text


def preprocess(text: str):
    # ссылки
    pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    text = re.sub(pattern, "", text)
    # числа
    text = re.sub(r"\d+", replace_number, text)
    # замена слов
    replacements = {
        r"\bд\b": "дом",
        r"\bкв\b": "квартира",
        r"\bул\b": "улица",
        r"\bк\b": "квартира",
    }
    text = replace_several_words(text, replacements)
    # удалить все символы кроме слов
    text = re.sub(r"[_\W]+", " ", text)
    # удалить все слова кроме русских
    text = re.sub(r"[^а-яА-ЯёЁ\s]", "", text)
    return text

In [22]:
preprocessed_corpus = df_corpus.apply(preprocess)
preprocessed_corpus.head()

0                                       снег на дороге
1                  очистить кабельный киоск от рекламы
2    Просим убрать все деревья и кустарники которые...
3    Неудовлетворительное состояние парадной надпис...
4                                             Граффити
Name: corpus, dtype: object

## Токенизация

In [23]:
def tokenize(text: str) -> list[str]:
    return word_tokenize(text, language="russian")

In [24]:
tokenized_corpus = preprocessed_corpus.apply(tokenize)
tokenized_corpus[:5]

0                                   [снег, на, дороге]
1            [очистить, кабельный, киоск, от, рекламы]
2    [Просим, убрать, все, деревья, и, кустарники, ...
3    [Неудовлетворительное, состояние, парадной, на...
4                                           [Граффити]
Name: corpus, dtype: object

## Нормализация

In [25]:
morph = MorphAnalyzer(lang="ru")
threshold = 0.45


def normalize(text: list[str]) -> list[str]:
    changed_text = []
    for word in text:
        parse = morph.parse(word)[0]
        if parse.score >= threshold:
            changed_text.append(parse.normal_form)
    return changed_text

In [26]:
normalized_corpus = tokenized_corpus.apply(normalize)
normalized_corpus[:5]

0                                   [снег, на, дорога]
1            [очистить, кабельный, киоск, от, реклама]
2    [просить, убрать, всё, дерево, и, кустарник, к...
3          [неудовлетворительный, состояние, на, этаж]
4                                                   []
Name: corpus, dtype: object

## Стоп-слова

In [27]:
stop_words = stopwords.words("russian")


def delete_stopwords(text: list[str]) -> list[str]:
    return [word for word in text if word not in stop_words]

In [28]:
filtered_corpus = normalized_corpus.apply(delete_stopwords)
filtered_corpus[:5]

0                                       [снег, дорога]
1                [очистить, кабельный, киоск, реклама]
2    [просить, убрать, всё, дерево, кустарник, кото...
3              [неудовлетворительный, состояние, этаж]
4                                                   []
Name: corpus, dtype: object

## Удаление редкоупотребляемых слов

In [29]:
vocabulary = [word for doc in filtered_corpus for word in doc]
len(vocabulary)

513543

In [ ]:
freaquency = pd.Series()
for word in tqdm(set(vocabulary)):
    freaquency.loc[word] = vocabulary.count(word)

# freaquency.to_csv("../datasets/lab1-feature_extraction/freaquency.csv")

In [52]:
def delete_rare_words(doc: list[str], min_count: int = 5) -> list[str]:
    return list(filter(lambda word: freaquency.loc[word] >= min_count, doc))


In [65]:
corpus = filtered_corpus.apply(delete_rare_words, min_count=5)
display(corpus[:10], len(corpus))

0                                       [снег, дорога]
1                [очистить, кабельный, киоск, реклама]
2    [просить, убрать, всё, дерево, кустарник, кото...
3              [неудовлетворительный, состояние, этаж]
4                                                   []
5    [необходимо, проверить, законность, установка,...
6    [уборка, производиться, лестница, очень, грязн...
7                                              [мусор]
8    [отсутствовать, освещение, площадка, шесть, се...
9    [делать, благоустройство, никто, убирать, мусо...
Name: corpus, dtype: object

43140

In [66]:
corpus = corpus[corpus.apply(lambda doc: len(doc) >= 5)]
display(corpus[:10], len(corpus))

2     [просить, убрать, всё, дерево, кустарник, кото...
5     [необходимо, проверить, законность, установка,...
6     [уборка, производиться, лестница, очень, грязн...
8     [отсутствовать, освещение, площадка, шесть, се...
9     [делать, благоустройство, никто, убирать, мусо...
15    [пожалуйста, удалить, бетонный, арматура, газо...
17    [ланский, дом, двенадцать, квартира, парадный,...
18           [кривой, висеть, просьба, поправить, этаж]
19    [плохой, уборка, улица, мусор, тротуар, газон,...
23    [демонтаж, рекламный, вывеска, фасад, образова...
Name: corpus, dtype: object

29359

In [57]:
# corpus.apply(" ".join).to_csv("../datasets/lab1-feature_extraction/corpus.csv", index=False)

## Матрица контекстных эмбеддингов

In [8]:
vocabulary = sorted(set([word for doc in corpus for word in doc]))
display(vocabulary[:10], len(vocabulary))

['абп',
 'абрамов',
 'абсолютно',
 'авангардный',
 'аварийка',
 'аварийность',
 'аварийный',
 'авария',
 'август',
 'авиаконструктор']

5190

In [67]:
N = len(vocabulary)
context_embeddings = pd.DataFrame(
    data=np.zeros((N, N)),
    index=vocabulary,
    columns=vocabulary,
    dtype=np.int16
)
context_embeddings

,абп,абрамов,абсолютно,авангардный,аварийка,аварийность,аварийный,авария,август,авиаконструктор,...,яндекс,яркий,ярко,ярмарка,ярослав,ярославский,ярус,яхтенный,ящик,ёлка
абп,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
абрамов,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
абсолютно,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
авангардный,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
аварийка,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ярославский,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ярус,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
яхтенный,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ящик,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for doc in tqdm(corpus):
    for i in range(len(doc) - 2):
        main_word = doc[i]
        for word in doc[i + 1 : i + 3]:
            context_embeddings.loc[main_word, word] += 1
            context_embeddings.loc[word, main_word] += 1

# context_embeddings.to_csv("../datasets/lab1-feature_extraction/context_embeddings.csv")

In [10]:
pca = PCA(n_components=200)
pca_embeddings = pca.fit_transform(context_embeddings)
w2v = pd.DataFrame(pca_embeddings, index=context_embeddings.index)
w2v.index.name = "word"
w2v

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
word,,,,,,,,,,,,,,,,,,,,,
абп,-13.745661,6.144555,1.672390,-0.837235,-3.745957,-0.866193,-0.170243,0.518571,1.121641,-1.053889,...,-0.124310,-0.076910,0.048945,0.031750,0.007667,-0.038009,0.168864,0.000495,-0.130860,0.141327
абрамов,-3.991519,1.339566,11.473537,-3.064182,-5.942729,-3.422678,2.007564,2.223933,2.904329,-3.163823,...,-1.809141,-0.098477,-0.810195,-0.452103,-0.543618,1.680041,0.453972,-0.655151,-0.514539,-0.426249
абсолютно,-11.821876,2.943006,-1.280718,-0.987937,-1.371568,-1.522340,-0.233250,-0.602862,2.170925,-0.611029,...,0.308653,0.359587,0.387458,-0.257858,0.229763,-0.563574,0.312862,-0.277851,0.412213,-0.883354
авангардный,-7.512660,4.299288,8.587495,-1.755458,-5.053628,1.171524,0.330959,1.959238,3.094127,0.620284,...,-0.382019,0.042043,-0.119812,-0.030248,-0.457969,0.764405,0.292025,-0.466514,-0.159710,-0.276336
аварийка,-12.834715,5.345289,1.413563,-0.377485,-3.892651,-0.032718,0.037899,0.643811,1.148384,-1.069906,...,-0.308458,-0.167686,0.190215,0.444776,-0.127919,0.217852,0.118530,-0.164167,0.000987,-0.000986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ярославский,-13.313072,6.173671,1.877300,-1.076308,-3.661942,-0.843670,-0.222866,0.142208,1.326214,-0.887128,...,0.078221,-0.057317,-0.069204,0.072322,-0.108619,-0.022130,0.102530,-0.009779,-0.023583,-0.048692
ярус,-13.842405,5.429180,-0.656739,-1.208449,-4.329737,-1.969817,-2.195102,-0.043728,0.903244,-0.638368,...,0.127138,-0.084120,0.125608,-0.524010,0.073769,0.107086,0.562880,0.067568,-0.627302,0.082190
яхтенный,-8.942939,4.312625,6.144215,-2.528917,-5.094325,-3.421365,0.679796,2.320170,2.383721,-0.365530,...,-0.387536,-0.467000,-0.279117,0.079510,-0.328350,0.893941,0.662707,-0.504196,-0.241658,0.203533


In [14]:
# w2v.to_csv("../datasets/lab1-feature_extraction/compressed_embeddings200.tsv", sep="\t")
# w2v.to_csv("../datasets/lab1-feature_extraction/compressed_embeddings200.csv")